In [1]:
# importing libraries
import pandas as pd
import os
import xlrd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from wordcloud import WordCloud, STOPWORDS
import numpy as np
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from pywsd.utils import lemmatize_sentence
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/raghavagovil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Warming up PyWSD (takes ~10 secs)... took 11.499523162841797 secs.


In [2]:
# reading in the data 
pd.set_option('display.max_columns', 500)
tweets = pd.read_csv('kevin_40k.csv', encoding = 'cp1252')
tweets = tweets[['text', 'created_at']]
tweets.head()

,text,created_at
0,Only taxation and ri0t mongering is their expe...,Sat Mar 21 07:06:21 +0000 2020
1,I like my women like I like my Coronavirus cur...,Sat Mar 21 07:06:26 +0000 2020
2,Scammers may use fake emails or texts to get y...,Sat Mar 21 07:06:27 +0000 2020
3,"""Map shows locations of airports where TSA off...",Sat Mar 21 07:06:27 +0000 2020
4,"""Homeschooling experts offer advice for parent...",Sat Mar 21 07:06:27 +0000 2020


In [3]:
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence) 
    return score

In [4]:
scores = []

for i in range(tweets.shape[0]): 
    scores.append(sentiment_analyzer_scores(str(tweets.iloc[i,])))

In [5]:
scores[0:10]

[{'neg': 0.087, 'neu': 0.913, 'pos': 0.0, 'compound': -0.2023},
 {'neg': 0.0, 'neu': 0.717, 'pos': 0.283, 'compound': 0.7579},
 {'neg': 0.129, 'neu': 0.871, 'pos': 0.0, 'compound': -0.4767},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.872, 'pos': 0.128, 'compound': 0.4215}]

In [9]:
# converting list to df
scores_df = pd.DataFrame(scores)

In [10]:
# final tweets df with sentiment scores
tweets = pd.concat([tweets.reset_index(drop=True), scores_df], axis=1)
tweets.head()

,text,created_at,neg,neu,pos,compound
0,Only taxation and ri0t mongering is their expe...,Sat Mar 21 07:06:21 +0000 2020,0.087,0.913,0.000,-0.2023
1,I like my women like I like my Coronavirus cur...,Sat Mar 21 07:06:26 +0000 2020,0.000,0.717,0.283,0.7579
2,Scammers may use fake emails or texts to get y...,Sat Mar 21 07:06:27 +0000 2020,0.129,0.871,0.000,-0.4767
3,"""Map shows locations of airports where TSA off...",Sat Mar 21 07:06:27 +0000 2020,0.000,1.000,0.000,0.0000
4,"""Homeschooling experts offer advice for parent...",Sat Mar 21 07:06:27 +0000 2020,0.000,1.000,0.000,0.0000


In [14]:
# finding phrases in text
donald_trump = sum(tweets['text'].str.contains('donald trump', case = False).astype(int))
fake_news = sum(tweets['text'].str.contains('fake news', case = False).astype(int))
united_states = sum(tweets['text'].str.contains('united_states', case = False).astype(int))
news = sum(tweets['text'].str.contains('news', case = False).astype(int))
media = sum(tweets['text'].str.contains('media', case = False).astype(int))

146